<a href="https://colab.research.google.com/github/david-garza/final_project/blob/bianca_1/pre_cleaned_data_to_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL and Save to CSV 
In order to have some pre cleaned data to work with on Tableau.

# ETL

## Beach Attributes File

In [61]:
# Import dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2
from datetime import datetime
from getpass import getpass
import time

In [62]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
# Bianca's path - Read beach_attributes from local gdrive location, MUST BE CHANGED TO MATCH YOUR FILE LOCATION!!!
beach_attr_df = pd.read_csv("/content/drive/MyDrive/Data/Beach/Beach_Attributes.csv")

In [64]:
# Examen the number of unique beaches in the data set
print(beach_attr_df["Beach Name"].unique())

# How many beaches are in galveston?
print(len(beach_attr_df["Beach Name"].unique()))

['25th St.' '45th St.' '61st St.' 'Appfel Park' 'Caplen'
 'Caplen/Crystal Beach' 'Clara St.' 'Dellanera Park' 'East Beach'
 'Erman Pilsner Boat Ramp' 'Frank Carmona Beach'
 'Galveston Island State Park' 'Galveston Island State Park Backside'
 'Gilchrist East' 'Gilchrist West' 'Gulf Shores' 'Haney Park'
 'High Island East' 'High Island West' 'Holiday' 'Indian Beach'
 'Jamaica Beach' "O'Neil Rd." 'Pirates Beach' 'Rettilon Road'
 'Rollover Pass East' 'Rollover Pass West' 'San Luis Pass' 'Sea Isle'
 'Seadrift' 'Skyline Drive' 'Skyline Park' 'Spanish Grant/Bermuda Beach'
 'Stewart Beach' 'Texas City Dike' 'West End']
36


In [65]:
# Investigate the number of unique values for each data column, columns with 1 value provide no information and can be dropped
beach_attr_df.nunique()

State                                    1
Year                                    16
Beach ID                                36
Beach Name                              36
County                                   1
Beach Ownership/Beach Accessibility      1
BeachLength (Mi)                        69
Tier                                     3
Start Latitude                          36
Start Longitude                         36
End Latitude                            36
End Longitude                           36
Waterbody Name                           1
Waterbody Type                           2
AdvisoryReportingFrequency(to EPA)       1
AdvisoryReportingFrequencyUnits          1
dtype: int64

In [66]:
# Investigate water type column, that is more then 36 beaches!
beach_attr_df["Waterbody Type"].value_counts()

Open Coast              469
Sound, Bay, or Inlet     75
Name: Waterbody Type, dtype: int64

In [67]:
# Number of years that beaches were reported
beach_attr_df["Year"].unique()

array([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012,
       2011, 2010, 2009, 2008, 2007])

In [68]:
# Create a list of columns that should be dropped, columns that have the same value
beach_attr_columns_to_drop = beach_attr_df.nunique()[beach_attr_df.nunique()==1].index
beach_attr_columns_to_drop

Index(['State', 'County', 'Beach Ownership/Beach Accessibility ',
       'Waterbody Name', 'AdvisoryReportingFrequency(to EPA)',
       'AdvisoryReportingFrequencyUnits'],
      dtype='object')

In [69]:
# Drop the same value columns
beach_attr_df.drop(beach_attr_columns_to_drop,1,inplace=True)
beach_attr_df.head(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,Year,Beach ID,Beach Name,BeachLength (Mi),Tier,Start Latitude,Start Longitude,End Latitude,End Longitude,Waterbody Type
0,2022,TX710697,25th St.,1.80,1,29.298146,-94.777565,29.284662,-94.794776,Open Coast
1,2022,TX214299,45th St.,1.55,1,29.284667,-94.794770,29.271917,-94.815865,Open Coast


In [70]:
# Reporting beaches are added again each year, so drop year. we only care about the beach information
beach_attr_df.drop("Year",1,inplace=True)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [71]:
# Determine the size and shape of the current dataframe after the year drop
print(beach_attr_df.shape)
beach_attr_df.head(2)

(544, 9)


,Beach ID,Beach Name,BeachLength (Mi),Tier,Start Latitude,Start Longitude,End Latitude,End Longitude,Waterbody Type
0,TX710697,25th St.,1.80,1,29.298146,-94.777565,29.284662,-94.794776,Open Coast
1,TX214299,45th St.,1.55,1,29.284667,-94.794770,29.271917,-94.815865,Open Coast


In [72]:
# Now that year is gone, we can drop the duplicate rows, and should only have the unique beaches left
beach_attr_df.drop_duplicates(inplace=True)
beach_attr_df.shape

(71, 9)

In [73]:
# There are 71 rows of data, but there are only 36 beaches on Galveston Island! Something is wrong!
len(beach_attr_df["Beach ID"].unique())

36

In [74]:
# Count how often a unique beach ID appears in the current data frame
beach_attr_df["Beach ID"].value_counts()

TX710697    2
TX822495    2
TX974690    2
TX669225    2
TX751320    2
TX832087    2
TX284256    2
TX341767    2
TX767833    2
TX593160    2
TX236175    2
TX406100    2
TX650612    2
TX163187    2
TX451421    2
TX164090    2
TX239942    2
TX272864    2
TX168454    2
TX669733    2
TX486021    2
TX327206    2
TX940700    2
TX972853    2
TX392019    2
TX393353    2
TX981462    2
TX426780    2
TX795287    2
TX334226    2
TX226514    2
TX568423    2
TX122528    2
TX860495    2
TX632189    2
TX214299    1
Name: Beach ID, dtype: int64

In [75]:
# Many beaches are still appearing twice in the data, why? Filter and display one beach ID to determine what is going on
beach_attr_df[beach_attr_df["Beach ID"]=="TX710697"]

,Beach ID,Beach Name,BeachLength (Mi),Tier,Start Latitude,Start Longitude,End Latitude,End Longitude,Waterbody Type
0,TX710697,25th St.,1.800,1,29.298146,-94.777565,29.284662,-94.794776,Open Coast
468,TX710697,25th St.,1.396,1,29.298146,-94.777565,29.284662,-94.794776,Open Coast


In [76]:
# The beach that is returned is the beach on 25th Street. Upon inspection, all the data is the same except for the BeachLength column. Fix this by dropping the beachlengthy, beaches should be the same length year after year
beach_attr_df.drop("BeachLength (Mi)",1,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [77]:
# Now that beachlength is dropped, the data should not have duplicates. Run drop_duplicates to remove the repeated rows of data
beach_attr_df.drop_duplicates(inplace=True)

In [78]:
# Now verify that each Beach ID only has one record
beach_attr_df["Beach ID"].value_counts()

TX710697    1
TX214299    1
TX239942    1
TX974690    1
TX669225    1
TX751320    1
TX832087    1
TX284256    1
TX341767    1
TX822495    1
TX767833    1
TX236175    1
TX406100    1
TX650612    1
TX163187    1
TX451421    1
TX164090    1
TX593160    1
TX272864    1
TX669733    1
TX981462    1
TX486021    1
TX327206    1
TX940700    1
TX972853    1
TX392019    1
TX393353    1
TX168454    1
TX632189    1
TX795287    1
TX334226    1
TX226514    1
TX568423    1
TX122528    1
TX860495    1
TX426780    1
Name: Beach ID, dtype: int64

In [79]:
# Check uniqueness of remaining columns
beach_attr_df.nunique()

Beach ID           36
Beach Name         36
Tier                3
Start Latitude     36
Start Longitude    36
End Latitude       36
End Longitude      36
Waterbody Type      2
dtype: int64

In [80]:
# Need to look up what is a Tier 1, 2, 3 beach. Print out the counts of the beach below
beach_attr_df["Tier"].value_counts()

1    23
2     7
3     6
Name: Tier, dtype: int64

In [81]:
# Check the number of different water types and what are they
beach_attr_df["Waterbody Type"].value_counts()

Open Coast              31
Sound, Bay, or Inlet     5
Name: Waterbody Type, dtype: int64

In [82]:
# Final table for load into a database. For database load, change column names, check data types match SQL data types, load into database table
beach_attr_df.head(2)

,Beach ID,Beach Name,Tier,Start Latitude,Start Longitude,End Latitude,End Longitude,Waterbody Type
0,TX710697,25th St.,1,29.298146,-94.777565,29.284662,-94.794776,Open Coast
1,TX214299,45th St.,1,29.284667,-94.794770,29.271917,-94.815865,Open Coast


In [83]:
# Final row and column count
beach_attr_df.shape

(36, 8)

In [84]:
# check the datatypes
beach_attr_df.dtypes

Beach ID            object
Beach Name          object
Tier                 int64
Start Latitude     float64
Start Longitude    float64
End Latitude       float64
End Longitude      float64
Waterbody Type      object
dtype: object

### ETL in preparation for loading the data into the Database

In [85]:
# Renaming the columns to match the database schema.
renamed_beach_attr_df = beach_attr_df.rename(columns={"Beach ID": "beach_id", "Beach Name":"beach_name", "Tier":"tier", "Start Latitude": "start_lat",
                                                      "Start Longitude": "start_long", "End Latitude":"end_lat", "End Longitude":"end_long", "Waterbody Type":"waterbody_type"})

renamed_beach_attr_df.head(2)

,beach_id,beach_name,tier,start_lat,start_long,end_lat,end_long,waterbody_type
0,TX710697,25th St.,1,29.298146,-94.777565,29.284662,-94.794776,Open Coast
1,TX214299,45th St.,1,29.284667,-94.794770,29.271917,-94.815865,Open Coast


## Water Quality File

In [86]:
# Bianca's path loading the data.
water_quality_df = pd.read_csv("/content/drive/MyDrive/Data/Beach/Water_Quality_Report.csv")
water_quality_df.head(2)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,State Code,Year,Beach ID(Def.),Beach Name(Def.),StationID(Def.),Station Name (Def.),CountyName(Def.),Identifier(Def.),StartDate(Def.),StartTime(Def.),...,ActivityDepthValue(Def.),ActivityDepthUnitCode(Def.),ResultAnalyticalMethodIdentifier Context(Def.),ResultAnalyticalMethodIdentifier(Def.),ResultAnalyticalMethodName(Def.),Sample CollectionMethod Identifier(Def.),Sample Collection Method Name(Def.),Field Gear(Def.),AnalysisDateTime(Def.),DetectionQuantitationLimit(Def.)
0,TX,2021,TX974690,Jamaica Beach,GAL014,Jamaica Beach South,GALVESTON,GAL014_20211227_90752,27-DEC-21,09:00:00 AM,...,2,ft,21TXBCH,19299,ENTEROLERT,SPL_WTRGRB,Sample Water Grab,Water Bottle,NaN,-
1,TX,2021,TX974690,Jamaica Beach,GAL014,Jamaica Beach South,GALVESTON,GAL014_20211227_90752,27-DEC-21,09:00:00 AM,...,2,ft,21TXBCH,19299,ENTEROLERT,SPL_WTRGRB,Sample Water Grab,Water Bottle,NaN,-


In [87]:
# Calculate number of rows and columns of raw data frame
water_quality_df.shape

(42540, 27)

In [88]:
# Deterine which columns have no data, same value, no values, etc
water_quality_df.nunique()

State Code                                            1
Year                                                 15
Beach ID(Def.)                                       23
Beach Name(Def.)                                     23
StationID(Def.)                                      52
Station Name (Def.)                                  52
CountyName(Def.)                                      1
Identifier(Def.)                                  28575
StartDate(Def.)                                    1988
StartTime(Def.)                                      41
ZoneCode(Def.)                                        2
ActivityTypeCode(Def.)                                1
CharacteristicName(Def.)                              1
Result ValueText(Def.)                                0
ResultValue(Def.)                                  3006
ResultMeasureUnit                                     3
Result Comment(Def.)                                  0
ActivityDepthValue(Def.)                        

In [89]:
# Create a list of the columns that have no useful data
water_quality_columns_to_drop =water_quality_df.nunique()[(water_quality_df.nunique()==1) | (water_quality_df.nunique()==0)].index

In [90]:
# Drop the useless data columns from the data frame
water_quality_df.drop(water_quality_columns_to_drop,1,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [91]:
# Review the new data frame. This is the data frame to load into database. Still needs, update column names, format StartDate to standard format.
water_quality_df.head(2)

,Year,Beach ID(Def.),Beach Name(Def.),StationID(Def.),Station Name (Def.),Identifier(Def.),StartDate(Def.),StartTime(Def.),ZoneCode(Def.),ResultValue(Def.),ResultMeasureUnit,ResultAnalyticalMethodIdentifier(Def.),ResultAnalyticalMethodName(Def.)
0,2021,TX974690,Jamaica Beach,GAL014,Jamaica Beach South,GAL014_20211227_90752,27-DEC-21,09:00:00 AM,CDT,7.94,MPN/100ml,19299,ENTEROLERT
1,2021,TX974690,Jamaica Beach,GAL014,Jamaica Beach South,GAL014_20211227_90752,27-DEC-21,09:00:00 AM,CDT,20.00,MPN/100ml,19299,ENTEROLERT


In [92]:
# Check the number of rows and columns of the "cleaned" data frame
water_quality_df.shape

(42540, 13)

In [93]:
# Inspect the number of unique values for each remaining column, inspect any that might be weird, shouldn't be unique etc.
water_quality_df.nunique()

Year                                         15
Beach ID(Def.)                               23
Beach Name(Def.)                             23
StationID(Def.)                              52
Station Name (Def.)                          52
Identifier(Def.)                          28575
StartDate(Def.)                            1988
StartTime(Def.)                              41
ZoneCode(Def.)                                2
ResultValue(Def.)                          3006
ResultMeasureUnit                             3
ResultAnalyticalMethodIdentifier(Def.)        5
ResultAnalyticalMethodName(Def.)              3
dtype: int64

In [94]:
# Examen start times.
water_quality_df["StartTime(Def.)"].value_counts()

09:00:00 AM    26510
08:00:00 AM     6431
07:00:00 AM     3892
06:00:00 AM     1944
10:00:00 AM     1236
08:30:00 AM      808
09:30:00 AM      653
11:00:00 AM      363
05:00:00 AM      286
07:30:00 AM      208
12:00:00 PM       80
10:30:00 AM       57
01:00:00 PM       21
06:30:00 AM        7
11:30:00 AM        6
07:40:00 AM        4
07:50:00 AM        3
02:00:00 PM        3
05:30:00 AM        3
10:15:00 AM        2
08:08:00 AM        2
03:00:00 PM        2
09:42:00 AM        1
09:45:00 AM        1
09:35:00 AM        1
09:33:00 AM        1
10:25:00 AM        1
09:06:00 AM        1
09:09:00 AM        1
09:26:00 AM        1
09:20:00 AM        1
09:15:00 AM        1
09:48:00 AM        1
09:03:00 AM        1
12:45:00 PM        1
09:51:00 AM        1
10:05:00 AM        1
10:07:00 AM        1
10:10:00 AM        1
10:02:00 AM        1
09:38:00 AM        1
Name: StartTime(Def.), dtype: int64

In [95]:
# These units need to be all the same. They many not be needed for model training. Keep for our records. Units are the same unit, see website in the next markdown cell for more information
water_quality_df["ResultMeasureUnit"].value_counts()

MPN/100ml    23186
cfu/100ml    16175
CFU           3179
Name: ResultMeasureUnit, dtype: int64

Note measurement units, they are the same units. https://www.researchgate.net/post/Is_there_a_way_to_convert_MPN_most_probable_number_to_CFU_colony_forming_unit#:~:text=The%20simple%20answer%20is%20that,is%20equal%20to%20one%20CFU.

In [96]:
# what is zoneCode? Appears to indicate if time is winter or summer time
water_quality_df["ZoneCode(Def.)"].value_counts()

CDT    36100
CST     6440
Name: ZoneCode(Def.), dtype: int64

### Converting necessary Datatypes 

In [97]:
# Convert the column 'StartDate(Def.)' from object to datetime.
water_quality_df["date"]= water_quality_df["StartDate(Def.)"].apply(lambda x:datetime.strptime(x,"%d-%b-%y"))

# Shift column "Date" to first position
first_column = water_quality_df.pop("date")

# Insert column using insert(position,column_name,first_column) function
water_quality_df.insert(0, "date", first_column)

# Drop the old column "StartDate(Def.)", which is now replaced by the column "date".
water_quality_df = water_quality_df.drop(columns=["StartDate(Def.)"])

water_quality_df.head(2)

,date,Year,Beach ID(Def.),Beach Name(Def.),StationID(Def.),Station Name (Def.),Identifier(Def.),StartTime(Def.),ZoneCode(Def.),ResultValue(Def.),ResultMeasureUnit,ResultAnalyticalMethodIdentifier(Def.),ResultAnalyticalMethodName(Def.)
0,2021-12-27,2021,TX974690,Jamaica Beach,GAL014,Jamaica Beach South,GAL014_20211227_90752,09:00:00 AM,CDT,7.94,MPN/100ml,19299,ENTEROLERT
1,2021-12-27,2021,TX974690,Jamaica Beach,GAL014,Jamaica Beach South,GAL014_20211227_90752,09:00:00 AM,CDT,20.00,MPN/100ml,19299,ENTEROLERT


In [98]:
# Check the datatypes to make sure the datime conversion was successful.
water_quality_df.dtypes

date                                      datetime64[ns]
Year                                               int64
Beach ID(Def.)                                    object
Beach Name(Def.)                                  object
StationID(Def.)                                   object
Station Name (Def.)                               object
Identifier(Def.)                                  object
StartTime(Def.)                                   object
ZoneCode(Def.)                                    object
ResultValue(Def.)                                float64
ResultMeasureUnit                                 object
ResultAnalyticalMethodIdentifier(Def.)            object
ResultAnalyticalMethodName(Def.)                  object
dtype: object

### Renaming Columns to match database schema.

In [99]:
# Renaming the columns to match the database schema.
renamed_water_quality_df = water_quality_df .rename(columns={"Year":"year", "Beach ID(Def.)": "beach_id", "Beach Name(Def.)": "beach_name",
                                                             "StationID(Def.)":"station_id", "Station Name (Def.)":"station_name",
                                                             "Identifier(Def.)":"identifier", "StartTime(Def.)":"start_time", "ZoneCode(Def.)":"zone_code",
                                                             "ResultValue(Def.)":"bacteria_count", "ResultMeasureUnit":"result_measure_unit",
                                                             "ResultAnalyticalMethodIdentifier(Def.)":"result_analytical_method_identifier",
                                                             "ResultAnalyticalMethodName(Def.)":"result_analytical_method_name"})
                                                         
                                                    

renamed_water_quality_df.head(2)

,date,year,beach_id,beach_name,station_id,station_name,identifier,start_time,zone_code,bacteria_count,result_measure_unit,result_analytical_method_identifier,result_analytical_method_name
0,2021-12-27,2021,TX974690,Jamaica Beach,GAL014,Jamaica Beach South,GAL014_20211227_90752,09:00:00 AM,CDT,7.94,MPN/100ml,19299,ENTEROLERT
1,2021-12-27,2021,TX974690,Jamaica Beach,GAL014,Jamaica Beach South,GAL014_20211227_90752,09:00:00 AM,CDT,20.00,MPN/100ml,19299,ENTEROLERT


# Main Weather Station file

In [100]:
# Bianca's path - Read beach_attributes from local gdrive location, MUST BE CHANGED TO MATCH YOUR FILE LOCATION!!!
weather_station1_df= pd.read_csv("/content/drive/MyDrive/Data/WX Station/USW00012923.csv",skiprows=1)
weather_station1_df.head(2)

,Date,TAVG (Degrees Fahrenheit),TMAX (Degrees Fahrenheit),TMIN (Degrees Fahrenheit),PRCP (Inches),SNOW (Inches),SNWD (Inches)
0,1946-08-01,NaN,86.0,77.0,0.0,0.0,0.0
1,1946-08-02,NaN,80.0,78.0,0.0,0.0,0.0


In [101]:
# Drop null values.
weather_station1_df.dropna()

,Date,TAVG (Degrees Fahrenheit),TMAX (Degrees Fahrenheit),TMIN (Degrees Fahrenheit),PRCP (Inches),SNOW (Inches),SNWD (Inches)
10684,2005-03-06,65.0,67.0,62.0,0.00,0.0,0.0
10685,2005-03-07,67.0,71.0,63.0,1.67,0.0,0.0
10686,2005-03-08,62.0,66.0,57.0,0.00,0.0,0.0
10687,2005-03-09,65.0,74.0,55.0,0.00,0.0,0.0
10688,2005-03-10,59.0,68.0,50.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...
16978,2022-05-31,86.0,91.0,83.0,0.00,0.0,0.0
16979,2022-06-01,86.0,90.0,81.0,0.00,0.0,0.0
16980,2022-06-02,85.0,93.0,77.0,0.59,0.0,0.0
16981,2022-06-03,83.0,90.0,78.0,0.00,0.0,0.0


In [102]:
# Check the data types on each column
weather_station1_df.dtypes

Date                          object
TAVG (Degrees Fahrenheit)    float64
TMAX (Degrees Fahrenheit)    float64
TMIN (Degrees Fahrenheit)    float64
PRCP (Inches)                float64
SNOW (Inches)                float64
SNWD (Inches)                float64
dtype: object

In [103]:
# Convert the column "Date" from object to datetime.
weather_station1_df["date"]= weather_station1_df["Date"].apply(lambda x:datetime.strptime(x,"%Y-%m-%d"))

# Shift column "Date" to first position
first_column = weather_station1_df.pop("date")

# Insert column using insert(position,column_name,first_column) function
weather_station1_df.insert(0, "date", first_column)

# Drop the old column "StartDate(Def.)", which is now replaced by the column "date".
weather_station1_df = weather_station1_df.drop(columns=["Date"])

weather_station1_df.head(2)

,date,TAVG (Degrees Fahrenheit),TMAX (Degrees Fahrenheit),TMIN (Degrees Fahrenheit),PRCP (Inches),SNOW (Inches),SNWD (Inches)
0,1946-08-01,NaN,86.0,77.0,0.0,0.0,0.0
1,1946-08-02,NaN,80.0,78.0,0.0,0.0,0.0


In [104]:
# Check the datatypes to make sure conversion happened.
weather_station1_df.dtypes

date                         datetime64[ns]
TAVG (Degrees Fahrenheit)           float64
TMAX (Degrees Fahrenheit)           float64
TMIN (Degrees Fahrenheit)           float64
PRCP (Inches)                       float64
SNOW (Inches)                       float64
SNWD (Inches)                       float64
dtype: object

In [105]:
# Renaming the columns to match the database schema.
renamed_weather_station1_df = weather_station1_df.rename(columns={"date":"date", "TAVG (Degrees Fahrenheit)":"avg_temp1", "TMAX (Degrees Fahrenheit)":"max_temp1",
                                                        "TMIN (Degrees Fahrenheit)":"min_temp1", "PRCP (Inches)":"precipitation1",
                                                        "SNOW (Inches)":"snowfall1", "SNWD (Inches)":"snow_depth1"})

renamed_weather_station1_df.head(2)

,date,avg_temp1,max_temp1,min_temp1,precipitation1,snowfall1,snow_depth1
0,1946-08-01,NaN,86.0,77.0,0.0,0.0,0.0
1,1946-08-02,NaN,80.0,78.0,0.0,0.0,0.0


# Save DFs to CSV files to be have cleaned data to explore on Tableau.


In [106]:
# save "renamed_beach_attr_df" to a CSV file.
renamed_beach_attr_df.to_csv('pre_cleaned_beach_attr.csv', index=False)

In [107]:
# save "renamed_water_quality_df" to a CSV file.
renamed_water_quality_df.to_csv('pre_cleaned_water_quality.csv', index=False)

In [108]:
# save "renamed_weather_station1" to a CSV file.
renamed_weather_station1_df.to_csv('pre_cleaned_weather_station1.csv', index=False)

# Merge dataframes 

In [109]:
# merging "renamed_water_quality_df" with "renamed_weather_station1"
merged_df = renamed_water_quality_df.merge(renamed_weather_station1_df, on='date', how='left')
merged_df.head(2)

,date,year,beach_id,beach_name,station_id,station_name,identifier,start_time,zone_code,bacteria_count,result_measure_unit,result_analytical_method_identifier,result_analytical_method_name,avg_temp1,max_temp1,min_temp1,precipitation1,snowfall1,snow_depth1
0,2021-12-27,2021,TX974690,Jamaica Beach,GAL014,Jamaica Beach South,GAL014_20211227_90752,09:00:00 AM,CDT,7.94,MPN/100ml,19299,ENTEROLERT,73.0,77.0,71.0,0.0,0.0,0.0
1,2021-12-27,2021,TX974690,Jamaica Beach,GAL014,Jamaica Beach South,GAL014_20211227_90752,09:00:00 AM,CDT,20.00,MPN/100ml,19299,ENTEROLERT,73.0,77.0,71.0,0.0,0.0,0.0


In [110]:
# merging "merged_df" with renamed_beach_attr_df

all_merged_df = renamed_beach_attr_df.merge(merged_df, on='beach_id', how='left')
all_merged_df.head(2)

,beach_id,beach_name_x,tier,start_lat,start_long,end_lat,end_long,waterbody_type,date,year,...,bacteria_count,result_measure_unit,result_analytical_method_identifier,result_analytical_method_name,avg_temp1,max_temp1,min_temp1,precipitation1,snowfall1,snow_depth1
0,TX710697,25th St.,1,29.298146,-94.777565,29.284662,-94.794776,Open Coast,2021-12-27,2021.0,...,5.0,MPN/100ml,19299,ENTEROLERT,73.0,77.0,71.0,0.0,0.0,0.0
1,TX710697,25th St.,1,29.298146,-94.777565,29.284662,-94.794776,Open Coast,2021-12-27,2021.0,...,5.0,MPN/100ml,19299,ENTEROLERT,73.0,77.0,71.0,0.0,0.0,0.0


In [111]:
# save to CSV merged dataframes.

# save "all_merged_df" to a CSV file.
renamed_water_quality_df.to_csv('pre_cleaned_and_merged_dfs.csv', index=False)